\# This notebook has steps for model initialisation and Training**



In [ ]:
!pip install transformers
!pip install tokenizer
!pip install datasets
!pip install rouge_score
!pip install sentencepiece
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f5ae896182deba952e311b6734d04af563a95213ab776c2644e1351810297aee
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from google.colab import drive
import pandas as pd
import textwrap
import json
import os
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
transformers.logging.set_verbosity_error()
from transformers import LEDForConditionalGeneration, LEDTokenizer
from datasets import load_dataset, load_metric
import torch
from rouge import Rouge
# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


**This section contains the code for loading the dataset and model initialization**

In [ ]:
#Load the datasets
dataset_path = "/content/drive/My Drive/Colab Notebooks/CS5242 Project/data/"

def load_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Join the paths
train_file_path = os.path.join(dataset_path, 'train_data.json')
test_file_path =  os.path.join(dataset_path, 'test_data.json')

# Load training data
train_data = load_data(train_file_path)

# Load testing data
test_data = load_data(test_file_path)


#Define the Model
class SummarizationModel:
    def __init__(self, model_name, device):
        self.model_name = model_name
        self.tokenizer = LEDTokenizer.from_pretrained(model_name)
        self.model = LEDForConditionalGeneration.from_pretrained(model_name).to(device)
        self.config=LEDForConditionalGeneration.from_pretrained(model_name).config

#Instantiate the model
model_name = "allenai/led-large-16384-arxiv"
model_summarizer = SummarizationModel(model_name, device=DEVICE)
model = model_summarizer.model
tokenizer=model_summarizer.tokenizer
# Define optimizer and criterion
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()
#print(modelsummarizer.config)

#

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

**Tokenize the sections before training the model**

In [ ]:
def process_section(section,results):

    # List to include all the sections, content and subsections and Subsection content
    summary_results = {}
    section_name = section.get("Section", "")
    content = section.get("Text", "")
    subsections = section.get("Subsections", [])
    ground_truth = section.get("Groundtruth", "")

    if not content and not ground_truth:
      return

    inputs = tokenizer(content, return_tensors="pt", max_length=1024, truncation=True)
    labels = tokenizer(ground_truth, return_tensors="pt", max_length=300, truncation=True)["input_ids"]

    # Append tokenized content and ground truth to the results
    results.append({"section_name": section_name, "inputs": inputs, "labels": labels})

    # Process the subsections if they exist
    if "Subsections" in section:
        for subsection in section["Subsections"]:
            process_section(subsection, results)
    return results


**Model Training**

In [ ]:
# Model Training
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0


    #print(train_loader)
    for data in train_loader:
      results = []
      process_section(data, results)
      for result in results:
        inputs = result["inputs"]
        labels = result["labels"]
        # Forward pass
        predictions = model(input_ids=inputs.input_ids.to(device), labels=labels.to(device))
        # Zero gradients
        optimizer.zero_grad()

        # Compute loss
        loss = predictions.loss

        # Backward pass

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

#Train the model
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)

num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader, optimizer, criterion, DEVICE)
    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}')

# Evaluation loop




Input ids are automatically padded from 2 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 32 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 40 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 350 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 17 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 881 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 160 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 25 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 197 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 252 to 1024 to be a multiple o

Epoch 1/10, Train Loss: 2.5874
Epoch 2/10, Train Loss: 1.8459
Epoch 3/10, Train Loss: 1.4828
Epoch 4/10, Train Loss: 1.1986
Epoch 5/10, Train Loss: 0.9783
Epoch 6/10, Train Loss: 0.7722
Epoch 7/10, Train Loss: 0.6453
